source of data: [Espérance de vie en 2024: Comparaisons régionales et départementales](https://www.insee.fr/fr/statistiques/2012749)<br>
[List of French departments by life expectancy](https://en.wikipedia.org/wiki/List_of_French_departments_by_life_expectancy) / 
[Продолжительность жизни в департаментах Франции](https://ru.wikipedia.org/wiki/Продолжительность_жизни_в_департаментах_Франции)<br />
[MapChart](https://www.mapchart.net/france.html)

In [2]:
import pandas as pd
from collections import namedtuple
import json

import sys
sys.path.append("..")
import mal_moduls_private.mal_total as mal

In [3]:
pd.options.display.min_rows = 6
pd.options.display.max_rows = 50

In [4]:
CREATE_LEGEND = True
YEAR = 2024

In [5]:
CountryGroup = namedtuple('CountryGroup', ['group_label', 'color', 'countries'])

In [6]:
df = pd.read_excel(f'data/France-{YEAR}.xlsx', sheet_name='REG', skiprows=4, skipfooter=1,
                   usecols="B:D", index_col='Unnamed: 1', na_values='nd')
df.rename(columns={'Hom.':'male', 'Fem.':'female'}, inplace=True)
df.head()

,male,female
Auvergne-Rhône-Alpes,81.0,86.1
Bourgogne-Franche-Comté,79.1,85.0
Bretagne,79.4,85.2
Centre-Val de Loire,79.4,85.2
Corse,81.6,85.8


In [7]:
df['ar_mean'] = ((df['female'] + df['male']) / 2).round(2)  # ! without rounding it is possible error during sorting
df.head()

,male,female,ar_mean
Auvergne-Rhône-Alpes,81.0,86.1,83.55
Bourgogne-Franche-Comté,79.1,85.0,82.05
Bretagne,79.4,85.2,82.30
Centre-Val de Loire,79.4,85.2,82.30
Corse,81.6,85.8,83.70


In [8]:
# pip install xlrd

<br />
<br />

In [10]:
df_metr = df.iloc[:13, :] \
            .sort_values(by=['ar_mean', 'male'], ascending=False)

df = pd.concat([df.loc[["France métropolitaine"], :], df_metr])

df

,male,female,ar_mean
France métropolitaine,80.1,85.7,82.90
Île-de-France,81.7,86.3,84.00
Corse,81.6,85.8,83.70
Auvergne-Rhône-Alpes,81.0,86.1,83.55
Provence-Alpes-Côte d'Azur,80.7,86.0,83.35
Pays de la Loire,80.2,86.0,83.10
Occitanie,80.3,85.7,83.00
Nouvelle-Aquitaine,79.9,85.4,82.65
Bretagne,79.4,85.2,82.30
Centre-Val de Loire,79.4,85.2,82.30


<br />
<br />

In [12]:
# just for interest, explore results: determine regions with max and min values, and also look at specific regions
# mal.min_and_max_values(df, row_center=["Provence-Alpes-Côte d'Azur", "France métropolitaine"], nmb=3, max_lng=9)

In [13]:
# Some inner statistics for interest:
def min_and_max(df, region_center=''):
    print(f"Number of values: {len(df)}")
    
    def find_3_largest(df):
        t_df = df.nlargest(3)
        t_df = t_df.reset_index()
        return t_df.apply(lambda v: f"{v.iloc[1]:.1f} – {v.iloc[0]}", axis=1)

    def find_3_smallest(df):
        t_df = df.nsmallest(3).iloc[::-1]
        t_df = t_df.reset_index()
        return t_df.apply(lambda v: f"{v.iloc[1]:.1f} – {v.iloc[0]}", axis=1)
    
    if region_center:
        ser_center = df.loc[region_center].map(lambda v: f" —— {v} ——")
        final_df = pd.concat([df.apply(find_3_largest), ser_center.to_frame().T, df.apply(find_3_smallest)])
        final_df.index = ['max', 'max_2', 'max_3', 'metr. France', 'min_3', 'min_2', 'min']
    else:
        final_df = pd.concat([df.apply(find_3_largest), df.apply(find_3_smallest)])
        final_df.index = ['max', 'max_2', 'max_3', 'min_3', 'min_2', 'min']
    return final_df.style.set_properties(**{'text-align': 'left'})

min_and_max(df, region_center='France métropolitaine')

Number of values: 14


,male,female,ar_mean
max,81.7 – Île-de-France,86.3 – Île-de-France,84.0 – Île-de-France
max_2,81.6 – Corse,86.1 – Auvergne-Rhône-Alpes,83.7 – Corse
max_3,81.0 – Auvergne-Rhône-Alpes,86.0 – Provence-Alpes-Côte d'Azur,83.5 – Auvergne-Rhône-Alpes
metr. France,—— 80.1 ——,—— 85.7 ——,—— 82.9 ——
min_3,79.1 – Bourgogne-Franche-Comté,84.7 – Normandie,82.0 – Grand Est
min_2,78.8 – Normandie,84.7 – Grand Est,81.8 – Normandie
min,78.1 – Hauts-de-France,84.1 – Hauts-de-France,81.1 – Hauts-de-France


In [14]:
# remove record about metropolitan France as a whole
df.drop(index=['France métropolitaine'], inplace=True)

<br />
<br />

In [16]:
# print(*df_metr.index.to_list(), sep='", "')

In [17]:
# renaming of provinces according to requirement of the online-service 'MapChart'
df.rename(index={
    "Île-de-France" : "Ile_de_France",
    "Auvergne-Rhône-Alpes" : "Auvergne_Rhone_Alpes",
    "Pays de la Loire" : "Pays_de_la_Loire",
    "Provence-Alpes-Côte d'Azur" : "Provence_Alpes_Cote_d_Azure",
    "Nouvelle-Aquitaine" : "Nouvelle_Aquitaine",
    "Centre-Val de Loire" : "Centre_Val_de_Loire",
    "Bourgogne-Franche-Comté" : "Bourgogne_Franche_Comte",
    "Grand Est" : "Grand_Est",
    "Hauts-de-France" : "Hauts_de_France",
}, inplace=True)

df.head(3)

,male,female,ar_mean
Ile_de_France,81.7,86.3,84.00
Corse,81.6,85.8,83.70
Auvergne_Rhone_Alpes,81.0,86.1,83.55


<br />
<br />
<br />

<h2> male </h2>

In [19]:
dd_legend_male = {
    '83.0–83.4' : '020f19',
    '82.5–82.9' : '062f4c',
    '82.0–82.4' : '08436e',
    '81.5–81.9' : '0b578d',
    '81.0–81.4' : '0d68a9',
    '80.5–80.9' : '107ac6',
    '80.0–80.4' : '128be1',
    '79.5–79.9' : '2e9eee',
    '79.0–79.4' : '51aef0',
    '78.5–78.9' : '6ebcf3',
    '78.0–78.4' : '92ccf6',
    '77.5–77.9' : 'b3dbf8',
    '77.0–77.4' : 'd4ebfb',
    '76.5–76.9' : 'edf7fd',
    '76.0–76.4' : 'f8fbfe'
}


def create_legend_code(dd_legend):
    for k, v in dd_legend.items():
        print(f"{{{{Legend|#{v}|{k}}}}}")  

if CREATE_LEGEND:
    create_legend_code(dd_legend_male)

{{Legend|#020f19|83.0–83.4}}
{{Legend|#062f4c|82.5–82.9}}
{{Legend|#08436e|82.0–82.4}}
{{Legend|#0b578d|81.5–81.9}}
{{Legend|#0d68a9|81.0–81.4}}
{{Legend|#107ac6|80.5–80.9}}
{{Legend|#128be1|80.0–80.4}}
{{Legend|#2e9eee|79.5–79.9}}
{{Legend|#51aef0|79.0–79.4}}
{{Legend|#6ebcf3|78.5–78.9}}
{{Legend|#92ccf6|78.0–78.4}}
{{Legend|#b3dbf8|77.5–77.9}}
{{Legend|#d4ebfb|77.0–77.4}}
{{Legend|#edf7fd|76.5–76.9}}
{{Legend|#f8fbfe|76.0–76.4}}


<br />
<br />

In [21]:
def filter_df(df, selected_col='male'):
    filtered_df = df.loc[:, [selected_col]]   \
                    .dropna()

    filtered_df['group_label'] = filtered_df[selected_col].map(lambda x: f"{x * 4 // 2 / 2:.1f}–{x * 4 // 2 / 2 + 0.4:.1f}")
    
    min_value = filtered_df[selected_col].min()
    max_value = filtered_df[selected_col].max()
    
    print(f"           ——— {selected_col} ———")
    print(f"Range: {min_value:.1f} – {max_value:.1f}   " +
          f"({filtered_df[selected_col].idxmin()} – {filtered_df[selected_col].idxmax()})")
    print(f"Number of groups: {filtered_df['group_label'].nunique()}")
    print(f"Number of values: {len(filtered_df)}")

    return filtered_df

df_male = filter_df(df, selected_col='male')
df_male

           ——— male ———
Range: 78.1 – 81.7   (Hauts_de_France – Ile_de_France)
Number of groups: 8
Number of values: 13


,male,group_label
Ile_de_France,81.7,81.5–81.9
Corse,81.6,81.5–81.9
Auvergne_Rhone_Alpes,81.0,81.0–81.4
Provence_Alpes_Cote_d_Azure,80.7,80.5–80.9
Pays_de_la_Loire,80.2,80.0–80.4
Occitanie,80.3,80.0–80.4
Nouvelle_Aquitaine,79.9,79.5–79.9
Bretagne,79.4,79.0–79.4
Centre_Val_de_Loire,79.4,79.0–79.4
Bourgogne_Franche_Comte,79.1,79.0–79.4


In [22]:
def extract_indexes(subdf, dd_legend=dd_legend_male):
    group_label = subdf['group_label'].iloc[0]
    countries = subdf.index.to_list()
    color = (dd_legend[group_label])
    
    ls_grouping.append(CountryGroup(group_label=group_label, countries=countries, color=color))

    return pd.Series([color, countries], index=['color', 'regions'])


ls_grouping = []
df_grouped = df_male.groupby(['group_label']).apply(extract_indexes, dd_legend=dd_legend_male).loc[::-1]

df_grouped

C:\Users\L3M\AppData\Local\Temp\ipykernel_38464\3406304120.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped = df_male.groupby(['group_label']).apply(extract_indexes, dd_legend=dd_legend_male).loc[::-1]


,color,regions
group_label,,
81.5–81.9,0b578d,"[Ile_de_France, Corse]"
81.0–81.4,0d68a9,[Auvergne_Rhone_Alpes]
80.5–80.9,107ac6,[Provence_Alpes_Cote_d_Azure]
80.0–80.4,128be1,"[Pays_de_la_Loire, Occitanie]"
79.5–79.9,2e9eee,[Nouvelle_Aquitaine]
79.0–79.4,51aef0,"[Bretagne, Centre_Val_de_Loire, Bourgogne_Fran..."
78.5–78.9,6ebcf3,[Normandie]
78.0–78.4,92ccf6,[Hauts_de_France]


In [23]:
def create_map_code(ls_grouping, title='', legend_color='#000'):  # (ls_grouping, title='', legend_color='#000'):
    false, true = False, True

    jo = {
        "groups": { },
        "title": title,
        "hidden": [],
        "background": "#ffffff",
        "borders": "#000",
        "legendFont": "Century Gothic",
        "legendFontColor": legend_color,
        "legendBorderColor": "#00000000",
        "legendBgColor": "#00000000",
        "legendWidth": 100,
        "legendBoxShape": "square",
        "areBordersShown": true,
        "defaultColor": "#d1dbdd",
        "labelsColor": "#6a0707",
        "labelsFont": "Arial",
        "strokeWidth": "medium",
        "areLabelsShown": true,
        "uncoloredScriptColor": "#ffff33",
        "v6": true,
        "legendPosition": "custom",
        "legendX": 120,
        "legendY": 500,
        "legendSize": "large",
        "legendTranslateX": "0.00",
        "legendStatus": "show",
        "scalingPatterns": true,
        "legendRowsSameColor": true,
        "legendColumnCount": 1
    }

    for group_label, color, regions in ls_grouping[::-1]:
        jo["groups"][f"#{color}"] = {"label": group_label,
                                     "paths": [department.replace(' ', '_') for department in regions]}       

    return jo


jo = create_map_code(ls_grouping, title=f'{YEAR}: ♂', legend_color='#333333')  # legend_color='#0000bf'

pretty_jo = json.dumps(jo, indent=2, ensure_ascii=False)

with open(f"output/map_JSON -{YEAR}, regions, male.txt", 'w', encoding="utf-8") as fh:
    fh.write(pretty_jo)

# print(pretty_jo)

<br />
<br />
<br />

<h2> female </h2>

In [25]:
dd_legend_female = {
    '88.0–88.4' : '1e0204',
    '87.5–87.9' : '60070f',
    '87.0–87.4' : 'a30d19',
    '86.5–86.9' : 'd81132',
    '86.0–86.4' : 'ef3b59',
    '85.5–85.9' : 'f2657d',
    '85.0–85.4' : 'f58698',
    '84.5–84.9' : 'f7a5b2',
    '84.0–84.4' : 'f9bfc6',
    '83.5–83.9' : 'fbd8db',
    '83.0–83.4' : 'fdebed',
    '82.5–82.9' : 'fef8f9'
}
 

if CREATE_LEGEND:
    create_legend_code(dd_legend_female)

{{Legend|#1e0204|88.0–88.4}}
{{Legend|#60070f|87.5–87.9}}
{{Legend|#a30d19|87.0–87.4}}
{{Legend|#d81132|86.5–86.9}}
{{Legend|#ef3b59|86.0–86.4}}
{{Legend|#f2657d|85.5–85.9}}
{{Legend|#f58698|85.0–85.4}}
{{Legend|#f7a5b2|84.5–84.9}}
{{Legend|#f9bfc6|84.0–84.4}}
{{Legend|#fbd8db|83.5–83.9}}
{{Legend|#fdebed|83.0–83.4}}
{{Legend|#fef8f9|82.5–82.9}}


In [26]:
df_female = filter_df(df, selected_col='female')
df_female

           ——— female ———
Range: 84.1 – 86.3   (Hauts_de_France – Ile_de_France)
Number of groups: 5
Number of values: 13


,female,group_label
Ile_de_France,86.3,86.0–86.4
Corse,85.8,85.5–85.9
Auvergne_Rhone_Alpes,86.1,86.0–86.4
Provence_Alpes_Cote_d_Azure,86.0,86.0–86.4
Pays_de_la_Loire,86.0,86.0–86.4
Occitanie,85.7,85.5–85.9
Nouvelle_Aquitaine,85.4,85.0–85.4
Bretagne,85.2,85.0–85.4
Centre_Val_de_Loire,85.2,85.0–85.4
Bourgogne_Franche_Comte,85.0,85.0–85.4


In [27]:
ls_grouping = []
df_grouped = df_female.groupby(['group_label']).apply(extract_indexes, dd_legend=dd_legend_female).loc[::-1]

df_grouped

C:\Users\L3M\AppData\Local\Temp\ipykernel_38464\2493027205.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped = df_female.groupby(['group_label']).apply(extract_indexes, dd_legend=dd_legend_female).loc[::-1]


,color,regions
group_label,,
86.0–86.4,ef3b59,"[Ile_de_France, Auvergne_Rhone_Alpes, Provence..."
85.5–85.9,f2657d,"[Corse, Occitanie]"
85.0–85.4,f58698,"[Nouvelle_Aquitaine, Bretagne, Centre_Val_de_L..."
84.5–84.9,f7a5b2,"[Grand_Est, Normandie]"
84.0–84.4,f9bfc6,[Hauts_de_France]


In [28]:
jo = create_map_code(ls_grouping, title=f'{YEAR}: ♀', legend_color='#333333')  # legend_color='#c20000'

pretty_jo = json.dumps(jo, indent=2, ensure_ascii=False)

with open(f"output/map_JSON -{YEAR}, regions, female.txt", 'w', encoding="utf-8") as fh:
    fh.write(pretty_jo)

# print(pretty_jo)

♂ <br />
♀